In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from keras.models import load_model, Sequential
from keras.layers import Dense, Activation
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.layers import Dense, Flatten
from keras.models import Model
import cv2
import cvlib as cv
from mtcnn import MTCNN
import glob
import pickle

In [165]:
def get_embedding(model, face_pixels):
	import numpy as np
	from sklearn.preprocessing import Normalizer
	face_pixels = face_pixels.astype('float32')
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	samples = np.expand_dims(face_pixels, axis=0)
	yhat = model.predict(samples)
	return yhat[0]

class AI_Retraining:
	def __init__(self, rotation_range = 60, horizontal_flip = True):
		from mtcnn import MTCNN
		from sklearn.preprocessing import LabelEncoder
		from keras.preprocessing.image import ImageDataGenerator

		self.datagen = ImageDataGenerator(rotation_range= rotation_range, horizontal_flip= horizontal_flip)
		self.face_detector = MTCNN()
		self.encoder = LabelEncoder()
		self.X_image = []
		self.y_label = []
		self.size = (160,160)
	def load_image(self, path):
		import cv2
		import glob
		try:
			self.path = path
			file_path = self.path
			for dirpath, dirnames, filenames in os.walk(file_path):
				for filename in filenames:
					i = file_path + "/" + filename
					img = cv2.imread(i)
					img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
					self.X_image.append(img)
					label = i.split("/")[-1]
					#label = label.split("//")[-1]
					label = label[:len(label)-4]
					self.y_label.append(label)
		except:
			print("Invalid Path")
	def extract_face(self):
		import cv2
		import os
		import pandas as pd
		X=[]
		y=[]
		try:
			if(os.path.isdir("C:/Users/DELL/Desktop/Kashmir/Kashmir Extracted Face") == True):
				for dirpath, dirnames, filenames in os.walk("C:/Users/DELL/Desktop/Kashmir/Kashmir Extracted Face"):
					for filename in filenames:
						i = "C:/Users/DELL/Desktop/Kashmir/Kashmir Extracted Face" + "/" + filename
						os.remove(i)
				os.rmdir("C:/Users/DELL/Desktop/Kashmir/Kashmir Extracted Face")
			self.extraction_path = "C:/Users/DELL/Desktop/Kashmir/Kashmir Extracted Face" 
			os.mkdir(self.extraction_path)
			temp = 0
			for i in range(len(self.X_image)):
				img = self.X_image[i]
				result = self.face_detector.detect_faces(img)
				k=0
				for k in range(len(result)):
					if(k >= 1):
						print("Required 1 person per image")
					else:
						x1, y1, width, height = result[k]['box']
						x1, y1 = abs(x1), abs(y1)
						x2, y2 = x1+width, y1+height
						face = img[y1:y2, x1:x2]
						face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
						face = cv2.resize(face,self.size)
						X.append(face)
						y.append(self.y_label[i])
						cv2.imwrite(self.extraction_path + "/{}.png".format(self.y_label[i]), face)
						temp += 1
			df = pd.DataFrame({"Image Number": list(range(temp)),"Name":y ,"No of Upsampling": ["NA"]*temp})
			df.to_csv(self.extraction_path+"/info.csv")
			print("Extraction Done")
			print(len(X))
			print(len(y))
		except Exception as e:
			print("code failed in extract_face")
			print(e)
	def face_augmentation(self, n = 7):
		import os
		import numpy as np
		import cv2
		try:
			if(os.path.isdir("C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image") == True):
				for dirpath, dirnames, filenames in os.walk("C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image"):
					for filename in filenames:
						i = "C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image" + "/" + filename
						os.remove(i)
				os.rmdir("C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image")
			self.augmentation_path = "C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image" 
			os.mkdir(self.augmentation_path)
			for i in range(len(self.X_image)):
				self.X_image[i] = cv2.resize(self.X_image[i],(160,160))
			for i in range(len(self.X_image)):
				img = self.X_image[i]
				img = np.reshape(img,(1,160,160,3))
				j=0
				for batch in self.datagen.flow(img, batch_size=1, save_to_dir= self.augmentation_path, save_prefix = self.y_label[i], save_format = "jpg"):
					if(j >= n):
						break
					j+=1
			return self
		except Exception as e:
			print("Code failed in face_augmentation")
			print(e)

	def load_augmented_image(self):
		import cv2
		X = []
		y_label = []
		try:
			for dirpath, dirnames, filenames in os.walk("C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image"):
					for filename in filenames:
						i = "C:/Users/DELL/Desktop/Kashmir/Kashmir Augmented Image" + "/" + filename
						img = cv2.imread(i)
						img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
						X.append(img)
						y = i.split("/")[-1]
						y = y.split("_")
						y = y[0] + "_" + y[1]
						y_label.append(y)
			self.X_image = X
			self.y_label = y_label
		except Exception as e:
			print("Code failed in load_augmented_image")
			print(e)

	def features_to_numbers(self,path):
		from keras.models import load_model
		from keras.utils import np_utils
		import pickle

		self.facenet_model = load_model(path)
		X = []
		y = []
		for i in range(len(self.X_image)):
			X.append(get_embedding(self.facenet_model, self.X_image[i]))
		y = self.encoder.fit_transform(self.y_label)
		pickle.dump(self.encoder,open("C:/Users/DELL/Desktop/encoder","wb"))
		self.no_classes = len(set(y))
		y = np_utils.to_categorical(y, self.no_classes)
		self.X_image = X
		self.y_label = y
	def training_NN(self, epochs = 10, path = r"C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model", threshold = 0.):
		from keras.models import Sequential
		from keras.layers import Dense
		import numpy as np
		import matplotlib.pyplot as plt
		import glob

		self.nn_model = Sequential()
		self.nn_model.add(Dense(self.no_classes*3, input_dim = 128 , activation = "relu"))
		self.nn_model.add(Dense(self.no_classes*2, activation = "relu"))
		self.nn_model.add(Dense(self.no_classes, activation = "softmax"))
		self.nn_model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics=["accuracy"])
		print(self.nn_model.summary())
		self.X_image = np.array(self.X_image)
		self.y_label = np.array(self.y_label)
		history = self.nn_model.fit(self.X_image, self.y_label, epochs = epochs)

		if(history.history["accuracy"][len(history.history["accuracy"])-1] > threshold and history.history["accuracy"][len(history.history["accuracy"])-1] != 1.0):
			if(os.path.isdir("C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model") == True):
				for dirpath, dirnames, filenames in os.walk("C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model/variables"):
					for filename in filenames:
						i = "C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model/variables" + "/" + filename
						os.remove(i)
				os.rmdir("C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model/variables")
				os.rmdir("C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model/assets")
				os.remove("C:/Users/DELL/Desktop/Kashmir Production/new_kashmir2.model/saved_model.pb")
			self.nn_model.save(path)
		else:
			print("Kindly Check the dataset is proper or update the model")


In [166]:
a = AI_Retraining()

In [167]:
a.load_image(r"D:/ASTRA/venv2/flask-admin-boilerplate-master/flask-admin-boilerplate-master/static/Trained_Images")

In [168]:
a.extract_face()

Required 1 person per image
Extraction Done
34
34


In [169]:
a.face_augmentation(3)

In [170]:
a.load_augmented_image()

In [171]:
a.features_to_numbers("C:/Users/DELL/Desktop/Kashmir/facenet_keras.h5/model/facenet_keras.h5")

In [172]:
a.training_NN(20)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 111)               14319     
_________________________________________________________________
dense_121 (Dense)            (None, 74)                8288      
_________________________________________________________________
dense_122 (Dense)            (None, 37)                2775      
Total params: 25,382
Trainable params: 25,382
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
148/148 [==============================] - 0s 1ms/step - loss: 3.6072 - accuracy: 0.0270
Epoch 2/20
148/148 [==============================] - 0s 116us/step - loss: 3.4520 - accuracy: 0.1216
Epoch 3/20
148/148 [==============================] - 0s 132us/step - loss: 3.3303 - accuracy: 0.2027
Epoch 4/20
148/148 [==============================] - 0s 132us/step - l